# ETL (Extrair, Transformar e Carregar) da camada Silver para Gold.

Este notebook realiza o processo de ETL para transformar e carregar os dados da camada Silver para a camada Gold no data lake. A camada Gold é otimizada para consultas analíticas e relatórios, garantindo que os dados estejam prontos para uso por ferramentas de BI e análise avançada.


### Configuração do Ambiente de Desenvolvimento

In [1]:
!pip install pyspark
!pip install psycopg2
!pip install pandas

# Criação e Carga das Dimensões (Star Schema)

Esta seção tem como objetivo realizar a **criação e carga das tabelas dimensão**
da camada Gold, seguindo o modelo **Star Schema**, a partir de dados previamente
tratados na camada Silver.


### Importação de Bibliotecas

Nesta seção são importadas as bibliotecas necessárias para:
- manipulação de dados com PySpark
- criação de colunas derivadas
- conexão com o banco PostgreSQL

In [2]:
import pandas as pd
import psycopg2

from psycopg2.extras import execute_batch
from psycopg2 import extras

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, concat_ws, year, month, dayofmonth, quarter, 
    dayofweek, when, date_format, monotonically_increasing_id,
    lit, trim, upper
)

from pyspark.sql import functions as F

from pyspark.sql.types import StringType, IntegerType, DateType, DecimalType

### Configuração do Ambiente

Nesta etapa são configurados:
- a sessão Spark
- a conexão JDBC com o PostgreSQL
- o schema de destino da camada Gold

In [3]:
spark = SparkSession.builder \
    .appName("SilverToGold") \
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")

POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_DB = "cat_db"
POSTGRES_USER = "admin"
POSTGRES_PASSWORD = "admin"

conn_params = {
    "host": POSTGRES_HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "port": POSTGRES_PORT
}

connection_properties = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}

try:
    conn = psycopg2.connect(**conn_params)
    
    query = "SELECT * FROM ACIDENTE"
    
    pdf = pd.read_sql(query, conn)
    df_silver = spark.createDataFrame(pdf)
    
    print("Dados da camada Silver carregados")
    df_silver.show()

except Exception as e:
    print(f"Erro na leitura: {e}")
finally:
    if conn:
        conn.close()
        
GOLD_SCHEMA = "gold"

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/22 22:37:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/tmp/ipykernel_51658/3971848863.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, conn)


Dados da camada Silver carregados


Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/home/yabamiah/Sandbox/cat-analytics/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
  File "/home/yabamiah/Sandbox/cat-analytics/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
    ~~~~~~~~~~~~~^^
BrokenPipeError: [Errno 32] Broken pipe
                                                                                

+------------------------+------------------------+--------------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+---------+-------------+---------------------+-----------------------+---------------+----------------+
|agente_causador_acidente|data_acidente_referencia|cbo_codigo_descricao|              cid_10|cnae_empregador|cnae_empregador_descricao|municipio_empregador|      natureza_lesao|parte_corpo_atingida|     sexo|tipo_acidente|uf_municipio_acidente|uf_municipio_empregador|data_nascimento|data_emissao_cat|
+------------------------+------------------------+--------------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+---------+-------------+---------------------+-----------------------+---------------+----------------+
|    Rua e Estrada - S...|              2023-03-27|       Administrador|   Frat da Clavicula| 

### Função Genérica de Carga de Dimensões

Esta função padroniza o processo de carga das dimensões, garantindo:
- remoção de duplicidades
- preservação da business key
- inserção no schema Gold
- validação básica pós-carga

In [4]:
def load_dim_with_psycopg2(table_name, pg_conn_params, spark):
    query = f"SELECT * FROM {table_name}"

    with psycopg2.connect(**pg_conn_params) as conn:
        pdf = pd.read_sql(query, conn)

    df_dim_loaded = spark.createDataFrame(pdf)
    return df_dim_loaded

In [5]:
def save_dimension(
    df_dim,
    dim_name,
    id_col_silver,
    other_cols,
    pg_conn_params,
    cols_to_drop=None
):
    """
    Salva dados em uma dimensão no PostgreSQL usando psycopg2,
    preservando a business key.
    Retorna um DataFrame Spark da dimensão carregada (com surrogate keys).
    """

    table_name = f"{GOLD_SCHEMA}.dim_{dim_name}"
    business_key_col = f"chv_{dim_name}_org"

    # Preparação da dimensão no Spark
    df_dim_unique = (
        df_dim
        .select(col(id_col_silver).alias(business_key_col), *other_cols)
        .distinct()
        .dropna(subset=[business_key_col])
    )

    if cols_to_drop:
        df_dim_unique = df_dim_unique.drop(*cols_to_drop)

    print(f"\n---> Criando e carregando Dimensão: {table_name}")
    count_unique = df_dim_unique.count()
    print(f"     Registros únicos: {count_unique}")

    if count_unique == 0:
        print("DataFrame vazio")
        return None

    # Converter para Pandas para inserção via psycopg2
    pdf = df_dim_unique.toPandas()

    columns = list(pdf.columns)
    cols_sql = ", ".join(columns)
    values_sql = ", ".join([f"%({c})s" for c in columns])

    insert_sql = f"""
        INSERT INTO {table_name} ({cols_sql})
        VALUES ({values_sql})
        ON CONFLICT ({business_key_col}) DO NOTHING
    """

    try:
        # Conexão PostgreSQL
        with psycopg2.connect(**pg_conn_params) as conn:
            with conn.cursor() as cur:
                execute_batch(
                    cur,
                    insert_sql,
                    pdf.to_dict(orient="records"),
                    page_size=1000
                )
            conn.commit()

        print("Inserção concluída")

        # Recarregar dimensão com surrogate keys via Spark
        df_dim_loaded = load_dim_with_psycopg2(
            table_name=table_name,
            pg_conn_params=pg_conn_params,
            spark=spark
        )


        count_check = df_dim_loaded.count()

        if count_check >= count_unique:
            print(f"Dimensão carregada. Registros confirmados: {count_check}")
        else:
            print(f"Confirmados ({count_check}) < esperados ({count_unique})")

        return df_dim_loaded

    except Exception as e:
        print(f"Erro ao gravar dimensão {table_name}: {e}")
        raise

### Dimensão Tempo

A dimensão tempo é utilizada para:
- data do acidente
- data de emissão da CAT
- data de nascimento do trabalhador

Ela permite análises temporais como sazonalidade, tendências e comparações anuais.

In [6]:
def create_dim_tempo(df_silver):    
    df_datas = df_silver.select("data_acidente_referencia").distinct() \
        .union(df_silver.select("data_emissao_cat").distinct()) \
        .union(df_silver.select("data_nascimento").distinct()) \
        .withColumnRenamed("data_acidente_referencia", "data") \
        .distinct() \
        .dropna()
    
    df_tempo = df_datas.select(
        col("data"),
        dayofmonth("data").alias("dia"),
        month("data").alias("mes"),
        date_format("data", "MMMM").alias("nome_mes"),
        quarter("data").alias("trimestre"),
        year("data").alias("ano"),
        dayofweek("data").alias("dia_semana"),
        when(dayofweek("data").isin(1, 7), True).otherwise(False).alias("is_fim_semana")
    )
    
    return save_dimension(
        df_tempo,
        "tmp",
        "data",
        ["dia", "mes", "nome_mes", "trimestre", "ano", "dia_semana", "is_fim_semana"],
        conn_params
    )

### Dimensão Trabalhador

Representa características demográficas e ocupacionais do trabalhador
no momento do acidente.

In [7]:
def create_dim_trabalhador(df_silver):
    df_trabalhador = df_silver.select(
        concat_ws("_", 
                  upper(trim(col("sexo"))),
                  col("cbo_codigo_descricao"),
                  col("data_nascimento")
        ).alias("srk_trb"),
        upper(trim(col("sexo"))).alias("sexo"),
        col("cbo_codigo_descricao").alias("srk_cbo"),
        col("data_nascimento").alias("srk_tmp_nsc")
    )
    
    return save_dimension(
        df_trabalhador,
        "trb",
        "srk_trb",
        ["sexo", "srk_cbo", "srk_tmp_nsc"],
        conn_params
    )

### Dimensão Empregador

A dimensão empregador representa as características da empresa
responsável pelo vínculo de trabalho no momento do acidente.

Esta dimensão referencia:
- a dimensão CNAE (atividade econômica)
- a dimensão Município (localização do empregador)

Sua criação ocorre após a carga das dimensões independentes,
garantindo integridade referencial.

In [8]:
def create_dim_empregador(df_silver):
    df_empregador = df_silver.select(
        concat_ws("_", 
                  col("cnae_empregador"),
                  col("municipio_empregador")
        ).alias("srk_emp"),
        col("cnae_empregador").alias("srk_cne"),
        col("municipio_empregador").alias("srk_mnc")
    )
    
    return save_dimension(
        df_empregador,
        "emp",
        "srk_emp",
        ["srk_cne", "srk_mnc"],
        conn_params
    )

### Dimensões de Classificação

As dimensões a seguir representam classificações oficiais utilizadas
para padronização e análise estatística.

In [9]:
def create_dim_cbo(df_silver):
    df_cbo = df_silver.select(
        col("cbo_codigo_descricao").alias("srk_cbo"),
        col("cbo_codigo_descricao").alias("codigo"),
        col("cbo_codigo_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cbo,
        "cbo",
        "srk_cbo",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cnae(df_silver):
    df_cnae = df_silver.select(
        col("cnae_empregador").alias("srk_cne"),
        col("cnae_empregador").alias("codigo"),
        col("cnae_empregador_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cnae,
        "cne",
        "srk_cne",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cid10(df_silver):
    df_cid = df_silver.select(
        col("cid_10").alias("srk_cid"),
        col("cid_10").alias("codigo"),
        col("cid_10").alias("descricao")
    )
    
    return save_dimension(
        df_cid,
        "cid",
        "srk_cid",
        ["codigo", "descricao"],
        conn_params
    )

### Dimensão Município

A dimensão município é utilizada para:
- local do acidente
- local do empregador

In [10]:
def create_dim_municipio(df_silver):
    df_mun_acidente = df_silver.select(
        col("uf_municipio_acidente").alias("codigo_ibge"),
        col("uf_municipio_acidente").alias("nome"),
        col("uf_municipio_acidente").alias("uf")
    )
    
    df_mun_empregador = df_silver.select(
        col("municipio_empregador").alias("codigo_ibge"),
        col("municipio_empregador").alias("nome"),
        col("uf_municipio_empregador").alias("uf")
    )
    
    df_municipio = df_mun_acidente.union(df_mun_empregador) \
        .distinct() \
        .dropna(subset=["codigo_ibge"])
    
    return save_dimension(
        df_municipio,
        "mnc",
        "codigo_ibge",
        ["nome", "uf"],
        conn_params
    )

### Dimensões de Caracterização do Acidente

Estas dimensões descrevem o contexto e as causas do acidente de trabalho.


In [11]:
def create_dim_tipo_acidente(df_silver):    
    df_tipo_acidente = df_silver.select(
        col("tipo_acidente").alias("srk_tpo_act"),
        col("tipo_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_tipo_acidente,
        "tpo_act",
        "srk_tpo_act",
        ["descricao"],
        conn_params
    )


def create_dim_lesao(df_silver):
    df_lesao = df_silver.select(
        concat_ws("_", 
                  col("natureza_lesao"), 
                  col("parte_corpo_atingida")
        ).alias("srk_lso"),
        col("natureza_lesao").alias("natureza_lesao"),
        col("parte_corpo_atingida").alias("parte_corpo_atingida")
    )
    
    return save_dimension(
        df_lesao,
        "lso",
        "srk_lso",
        ["natureza_lesao", "parte_corpo_atingida"],
        conn_params
    )


def create_dim_agente_causador(df_silver):
    df_agente = df_silver.select(
        col("agente_causador_acidente").alias("srk_agt_cdr"),
        col("agente_causador_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_agente,
        "agt_cdr",
        "srk_agt_cdr",
        ["descricao"],
        conn_params
    )

## Execução da Carga das Dimensões

Nesta etapa é executado o processo completo de criação e carga das dimensões,
respeitando a ordem de dependência entre elas.

In [12]:
try:
    dim_tempo = create_dim_tempo(df_silver)          
    dim_cbo = create_dim_cbo(df_silver)            
    dim_municipio = create_dim_municipio(df_silver)      
    dim_cnae = create_dim_cnae(df_silver)           
    dim_tipo_acidente = create_dim_tipo_acidente(df_silver)  
    dim_lesao = create_dim_lesao(df_silver)          
    dim_agente_causador = create_dim_agente_causador(df_silver)
    dim_cid10 = create_dim_cid10(df_silver)          

    dim_trabalhador = create_dim_trabalhador(df_silver)
    dim_empregador = create_dim_empregador(df_silver)
        
    print("Dimensões foram carregadas")
        
except Exception as e:
    print(f"\nERRO NA CARGA DAS DIMENSÕES: {e}")
    raise e


---> Criando e carregando Dimensão: gold.dim_tmp


     Registros únicos: 18083


Inserção concluída


/tmp/ipykernel_51658/2498911532.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, conn)


Dimensão carregada. Registros confirmados: 18083

---> Criando e carregando Dimensão: gold.dim_cbo


...

Dimensão carregada. Registros confirmados: 132473

---> Criando e carregando Dimensão: gold.dim_emp


     Registros únicos: 39659


Inserção concluída


/tmp/ipykernel_51658/2498911532.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, conn)


Dimensão carregada. Registros confirmados: 39659
Dimensões foram carregadas


### Preparando dados da camada Silver

In [13]:
from pyspark.sql.functions import col, concat_ws, upper, trim, year, lit

# Transformações iniciais e criação de chaves compostas
df_base = df_silver.select(
    # Chave SRK Categoria
    concat_ws("_",
        col("data_acidente_referencia"),
        col("sexo"),
        col("cbo_codigo_descricao"),
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("srk_cat"),
    
    # Chave SRK Trabalhador
    concat_ws("_", 
        upper(trim(col("sexo"))),
        col("cbo_codigo_descricao"),
        col("data_nascimento")
    ).alias("srk_trb"),

    # Chave SRK Empregador
    concat_ws("_", 
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("srk_emp"),

    # Chave composta para Lesão (Trazido da célula 2 para cá)
    concat_ws("_", 
        col("natureza_lesao"), 
        col("parte_corpo_atingida")
    ).alias("lesao_join_key"),

    # Renomeações diretas e colunas de dados
    col("data_acidente_referencia").alias("data_acidente"),
    col("data_emissao_cat").alias("data_emissao"),
    col("data_nascimento"),
    col("cbo_codigo_descricao").alias("codigo_cbo"),
    col("cnae_empregador").alias("codigo_cnae"),
    col("uf_municipio_acidente").alias("codigo_municipio_acidente"),
    col("municipio_empregador").alias("codigo_municipio_empregador"),
    col("tipo_acidente").alias("codigo_tipo_acidente"),
    col("agente_causador_acidente").alias("codigo_agente_causador"),
    col("cid_10").alias("codigo_cid10"),

    # Cálculo de idade
    (year(col("data_acidente_referencia")) - year(col("data_nascimento"))).cast("int").alias("idade_trabalhador")
)

print(f"Registros preparados: {df_base.count()}")

[Stage 131:=======>                                                 (1 + 7) / 8]

Registros preparados: 145888


## Lookup de Surrogate Keys

Realiza joins (lookups) entre df_base e todas as dimensões
para substituir business keys por surrogate keys (FKs).

- Trocar valores de negócio por IDs técnicos (surrogate keys)
- Criar relacionamentos entre fato e dimensões
- Preparar estrutura final da tabela fato

In [14]:
# Célula 2: Loop de Joins Corrigido

# Lista de configurações dos Joins
# (DataFrame Dimensão, Coluna Chave Dimensão, Coluna Chave Fato, Coluna SRK Original, Novo Nome SRK)
join_specs = [
    (dim_tempo, "chv_tmp_org", "data_acidente", "srk_tmp", "srk_tmp_act"),
    (dim_tempo, "chv_tmp_org", "data_emissao", "srk_tmp", "srk_tmp_ems"),
    (dim_tempo, "chv_tmp_org", "data_nascimento", "srk_tmp", "srk_tmp_nsc"),
    (dim_trabalhador, "chv_trb_org", "srk_trb", "srk_trb", "srk_trb"),
    (dim_cbo, "chv_cbo_org", "codigo_cbo", "srk_cbo", "srk_cbo"),
    (dim_empregador, "chv_emp_org", "srk_emp", "srk_emp", "srk_emp"),
    (dim_cnae, "chv_cne_org", "codigo_cnae", "srk_cne", "srk_cne"),
    (dim_municipio, "chv_mnc_org", "codigo_municipio_acidente", "srk_mnc", "srk_mnc_act"),
    (dim_municipio, "chv_mnc_org", "codigo_municipio_empregador", "srk_mnc", "srk_mnc_emp"),
    (dim_tipo_acidente, "chv_tpo_act_org", "codigo_tipo_acidente", "srk_tpo_act", "srk_tpo_act"),
    (dim_lesao, "chv_lso_org", "lesao_join_key", "srk_lso", "srk_lso"),
    (dim_agente_causador, "chv_agt_cdr_org", "codigo_agente_causador", "srk_agt_cdr", "srk_agt_cdr"),
    (dim_cid10, "chv_cid_org", "codigo_cid10", "srk_cid", "srk_cid")
]

df_fact = df_base

for dim_df, dim_key, fact_key, dim_srk, fact_alias in join_specs:
    
    # 1. Definimos um nome temporário para evitar colisão (Ambiguous Reference)
    # Isso acontece porque, por exemplo, 'srk_trb' existe tanto na fato quanto na dimensão
    temp_col_name = f"{fact_alias}_temp_new"
    
    lookup_dim = dim_df.select(
        col(dim_key).alias("join_key_dim"), 
        col(dim_srk).alias(temp_col_name) # Usamos o nome temporário aqui
    )
    
    df_fact = df_fact.join(
        lookup_dim,
        col(fact_key) == col("join_key_dim"),
        "left"
    )
    
    # 2. Removemos a chave auxiliar E a coluna original da fato (ex: removemos a srk_trb string antiga)
    df_fact = df_fact.drop("join_key_dim", fact_key)
    
    # 3. Renomeamos a coluna temporária para o nome final desejado
    df_fact = df_fact.withColumnRenamed(temp_col_name, fact_alias)

# Verificação opcional para garantir que não restaram colunas duplicadas
print("Schema após joins:")
df_fact.printSchema()

Schema após joins:
root
 |-- srk_cat: string (nullable = false)
 |-- idade_trabalhador: integer (nullable = true)
 |-- srk_tmp_act: long (nullable = true)
 |-- srk_tmp_ems: long (nullable = true)
 |-- srk_tmp_nsc: long (nullable = true)
 |-- srk_trb: long (nullable = true)
 |-- srk_cbo: long (nullable = true)
 |-- srk_emp: long (nullable = true)
 |-- srk_cne: long (nullable = true)
 |-- srk_mnc_act: long (nullable = true)
 |-- srk_mnc_emp: long (nullable = true)
 |-- srk_tpo_act: long (nullable = true)
 |-- srk_lso: long (nullable = true)
 |-- srk_agt_cdr: long (nullable = true)
 |-- srk_cid: long (nullable = true)



## Montagem da Tabela Fato

In [15]:
# Seleção final e reordenação das colunas
cols_final = [
    col("srk_cat").alias("chv_cat_org"),
    "srk_tmp_act", "srk_tmp_ems", "srk_tmp_nsc",
    "srk_trb", "srk_cbo", "srk_emp", "srk_cne",
    "srk_mnc_act", "srk_mnc_emp",
    "srk_tpo_act", "srk_lso", "srk_agt_cdr", "srk_cid",
    "idade_trabalhador"
]

df_fact_final = df_fact.select(*cols_final).distinct()

print(f"Registros na tabela fato: {df_fact_final.count()}")

[Stage 166:>                                                        (0 + 8) / 8]

Registros na tabela fato: 145870


### Carga da Tabela Fato

In [ ]:
colunas = df_fact_final.columns
fact_table_name = f"{GOLD_SCHEMA}.fat_act_trb"

print(f"Colunas: {len(colunas)}")
print(f"Colunas a inserir: {colunas}")
print(f"\nPreparando dados para inserção...")

dados_para_inserir = [tuple(row) for row in df_fact_final.collect()]

conn = None
cursor = None

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = f"INSERT INTO {fact_table_name} ({', '.join(colunas)}) VALUES %s"
    
    extras.execute_values(
        cursor,
        query,
        dados_para_inserir,
        template=None,
        page_size=1000
    )
    
    conn.commit()
    print("Registros concluída com sucesso!")
except Exception as e:
    print(f"\nErro ao inserir dados: {e}")
    if conn:
        conn.rollback()
    raise e
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    spark.stop()

Colunas: 15
Colunas a inserir: ['chv_cat_org', 'srk_tmp_act', 'srk_tmp_ems', 'srk_tmp_nsc', 'srk_trb', 'srk_cbo', 'srk_emp', 'srk_cne', 'srk_mnc_act', 'srk_mnc_emp', 'srk_tpo_act', 'srk_lso', 'srk_agt_cdr', 'srk_cid', 'idade_trabalhador']

Preparando dados para inserção...
